# Type Inference Using Unification

We will continue where we left off from the type inference lecture, talking about a systematic type inference engine for Lettuce. In this lecture, we will achieve the following: 
- Implement the informal algorithm from last time into a more formal procedure.
- Weave in constraint generation and constraint solving carefully to create a single type inference procedure that is capable of giving us better error messages.



## Grammar

We will use the following grammar that includes recursion. Note that we do not allow type annotations
in our version. All type annotations will need to be infered.

$$\begin{array}{rcll}
\mathbf{Program} & \rightarrow & TopLevel(\mathbf{Expr}) \\[5pt]
\mathbf{Expr} & \rightarrow & Const(\mathbf{Number}) \\
 & | & Ident(\mathbf{Identifier}) \\
 & | & Plus(\mathbf{Expr}, \mathbf{Expr}) \\
 & | & Mult(\mathbf{Expr}, \mathbf{Expr}) \\
 & | & Eq(\mathbf{Expr}, \mathbf{Expr}) \\
 & | & Geq (\mathbf{Expr}, \mathbf{Expr}) \\
 & | & IfThenElse(\mathbf{Expr}, \mathbf{Expr}, \mathbf{Expr}) & \text{if (expr) then expr else expr} \\
 & | & Let( \mathbf{Identifier}, \mathbf{Expr}, \mathbf{Expr}) & \text{let identifier = expr in expr} \\
 & | & FunDef( \mathbf{Identifier}, \mathbf{Expr}) & \text{function (identifier-formal-parameter) expr } \\ 
 & | & FunCall(\mathbf{Expr}, \mathbf{Expr}) & \text{function call - expr(expr)} \\
 & | & {LetRec(\mathbf{Identifier}, \mathbf{Identifier}, \mathbf{Expr}, \mathbf{Expr}) } & \text{argument 1 - function name, argument 2 - parameter}\\
 &&& \text{argument 3 - function definition expression, argument 4 - body expr} \\[5pt]
\end{array}$$


In [1]:
sealed trait Program
sealed trait Expr
case class Const(f: Double) extends Expr
case class Ident(s: String) extends Expr
case class Minus(e1: Expr, e2: Expr) extends Expr
case class Plus(e1: Expr, e2: Expr) extends Expr
case class Mult(e1: Expr, e2: Expr) extends Expr
case class Eq(e1: Expr, e2: Expr) extends Expr
case class Geq(e1: Expr, e2: Expr) extends Expr
case class IfThenElse(e1: Expr, e2: Expr, e3: Expr) extends Expr
case class Let(x: String, e1: Expr, e2: Expr) extends Expr
case class FunDef(id: String, e: Expr) extends Expr
case class FunCall(calledFun: Expr, argExpr: Expr) extends Expr
case class LetRec(funName: String, param: String, funExpr: Expr, bodyExpr: Expr) extends Expr

case class TopLevel(e: Expr) extends Program

defined trait Program
defined trait Expr
defined class Const
defined class Ident
defined class Minus
defined class Plus
defined class Mult
defined class Eq
defined class Geq
defined class IfThenElse
defined class Let
defined class FunDef
defined class FunCall
defined class LetRec
defined class TopLevel

Our goal is to infer types. Previously we worked as follows:

1. Generate a system of constraints first, and having all the constraints in place 
2. Solve the system of constraints using the substitution/unification approach we described.

Let us define an AST for our types along with some utilities. The AST is going to allow types to be a type variable. For convenience, we will override the `toString` method to have a pretty printer for our types.


In [2]:
sealed trait Type 

case object NumType extends Type {
    override def toString: String = "num"
}
case object BoolType extends Type {
    override def toString: String = "bool"
}
case class FunType(t1: Type, t2: Type) extends Type {
    override def toString: String = s"($t1 => $t2)"
}

// A type variable is now allowed since we are inferring types.
// The type variable has a number that identifies the variable uniquely.

case class TypeVar(j: Int) extends Type {
    override def toString: String = s"t$j"
}

/* --
TypeEquation: 

A type equation models an equation of the form t1 == t2
--*/

case class TypeEquation(t1: Type, t2: Type)

/*-- 
TypeConstraints: 
A class that contains mutables : 

   nTypeVars -- number of type variables so far
   l -- list of type equations added thus far.
--*/
case class TypeConstraints() {
    
    var nTypeVars: Int = 0
    
    var l: List[TypeEquation] = List()
    
    /* createFreshVar is a method that creates a fresh type variable. */
    def createFreshVar(): TypeVar = {
        val t = TypeVar(this.nTypeVars)
        nTypeVars = nTypeVars + 1
        return t
    }
    
    /* addTypeEquation is a method that adds a new type equation to our system */
    def addTypeEquation(t1: Type, t2: Type): Unit = {
        if (t1 == t2){
            // For convenience, we will suppress trivial equations
            println(s"Debug: trivial type equation $t1 == $t2 suppressed.")
        } else {
           l = TypeEquation(t1, t2)::l
        }
    }
    
    /* printAllTypeEquations pretty prints all the type equations thus far */
    def printAllTypeEquations(): Unit = {
        for (te <- l) {
            te match {
                case TypeEquation(t1, t2) => {println(s"$t1 == $t2")}
            }
        }
    }
}

defined trait Type
defined object NumType
defined object BoolType
defined class FunType
defined class TypeVar
defined class TypeEquation
defined class TypeConstraints

## 1. Generating Type Variables and Constraints from a Program

Let us focus on how to generate constraints from a program.  As mentioned previously, we will walk the program recursively and generate new type variables and constraints as needed.

Function `generateTypeVarsAndConstraints` has the following inputs:
- `e` is an expression that we are currently processing.
- `alpha` is a type environment: A map from Strings (identifier names) to their current types. Note that the type can invove type variables.
- `tCons` a TypeConstraints that has the current list of equations and new variables.

The output of the `generateTypeVarsAndConstraints` is a Type, which is the type of the expression `e` being input.



In [3]:
def generateTypeVarsAndConstraints(e: Expr, alpha: Map[String, Type], tCons: TypeConstraints): Type = {

    /* handleSubExprs is a utility method
     It recursively calls generateTypeVarsAndConstraints on e1 and e2 
     Each operand must equal operandType
     result equal to the resultType
     */
    def handleSubExprs(e1: Expr, e2: Expr, operandType: Type, resultType: Type): Type = {
        val t1 = generateTypeVarsAndConstraints(e1, alpha, tCons)
        /* ADD EQUATION t1 == desiredType */
        tCons.addTypeEquation(t1, operandType)
        val t2 = generateTypeVarsAndConstraints(e2, alpha, tCons)
        /* ADD EQUATION t2 == desiredType */
        tCons.addTypeEquation(t2, operandType)
        resultType
    }
    /* Now we walk the expression */
    e match {
        case Const(_) => NumType /* Numeric constants always have NumType */
        
        case Ident(s) => if (alpha contains s) 
                            alpha(s) /* Identifier types are just looked up from the environment, as usual */
                         else 
                            throw new IllegalArgumentException(s"Unknown identifier: $s")
        
        /* Let us use handleSubExprs to handle arithmtic and comparisons. 
        Note that the desired type is NumType */
        
        case Minus(e1, e2) => handleSubExprs(e1, e2, NumType, NumType)
        
        case Plus(e1, e2) => handleSubExprs(e1, e2, NumType, NumType)
        
        case Mult(e1, e2) => handleSubExprs(e1, e2, NumType, NumType)
        
        /* Note: Desired type of Geq is BoolType */
        
        case Geq(e1, e2) => handleSubExprs(e1, e2, NumType, BoolType)
        
        /* e1 == e2 is handled as follows: 
           generate type variables and constraints for e1
           generate type variables and constraints for e2 
           Add type equation: type(e1) == type(e2)
           */
        case Eq(e1, e2) => {
            val t1 = generateTypeVarsAndConstraints(e1, alpha, tCons)
            val t2 = generateTypeVarsAndConstraints(e2, alpha, tCons)
            /* EQUATION: t1 == t2 */
            tCons.addTypeEquation(t1, t2)
            /* Return type of Equality is Boolean */
            BoolType
        }
        
        case IfThenElse(e, e1, e2) => {
            /* First process the conditional expression e*/
            val t = generateTypeVarsAndConstraints(e, alpha, tCons)
            /* t must be boolean */
            tCons.addTypeEquation(t, BoolType)
            /* Next process the subexpressions e1, e2 */
            val t1 = generateTypeVarsAndConstraints(e1, alpha, tCons)
            val t2 = generateTypeVarsAndConstraints(e2, alpha, tCons)
            /* Equation: t1 == t2 */
            tCons.addTypeEquation(t1, t2)
            t1 /* May as well have been t2, but it does not matter for now */
        }
        
        case Let(x, e1, e2) => {
            /* For a let binding, first process e1 */
            val t1 = generateTypeVarsAndConstraints(e1, alpha, tCons)
            /* update the type environment to handle (x -> t1) */
            val newEnv = alpha + (x -> t1)
            /* process e2 */
            generateTypeVarsAndConstraints(e2, newEnv, tCons)
        }
        
        case FunCall(e1, e2) => {
            /* First process the function call expression t */
            val t = generateTypeVarsAndConstraints(e1, alpha, tCons)
            t match {
                /* t is explicitly a function type. 
                   Add equation that the type of the function call argument e2 must be the same as 
                   the type t takes for its input */
                case FunType(t1Hat , t2Hat) => {
                    val t1 = generateTypeVarsAndConstraints(e2, alpha, tCons)
                    /* Equation: t1Hat == t1 */
                    tCons.addTypeEquation(t1, t1Hat)
                    /* Type of call must be t2Hat */
                    t2Hat
                }
                case TypeVar(j) => {
                    /* t is a type variable. 
                       Generate a new type variable for the result of the call */
                    val nVar = tCons.createFreshVar() // Create a placeholder for the result type
                    println(s"Type Var: $e has type variable $nVar")
                    /* Generate the type of the argument of the call */
                    val t1 = generateTypeVarsAndConstraints(e2, alpha, tCons)
                    /* Now we note that the type of the function called: t must equal t1 => nVar */
                    val t3 = FunType(t1, nVar)
                    /* Equation: t == t1 => nVar */
                    tCons.addTypeEquation(t, t3)
                    /* return type is nVar */
                    nVar
                }
                case _ => throw new IllegalArgumentException(s"Type inference error: function call on non function type $t")
            }
        }
        
        case FunDef(x, e1) => {
            //1. Create a new variable for the formal argument x
            val tx = tCons.createFreshVar()
            println(s"Type Var: $x has type variable $tx")
            //2. Add it to the type environment
            val newEnv = alpha + (x -> tx)
            /* 3. Process the body of the function */
            val t2 = generateTypeVarsAndConstraints(e1, newEnv, tCons)
            //4. Type of this whole thing is tx => t2
            FunType(tx, t2)
        }
        
        case LetRec(funName, param, funExpr, bodyExpr) => {
            //1. Create a new variable for the formal argument x
            val tx = tCons.createFreshVar()
            println(s"Type Var: $param has type variable $tx")
            //2. Create a new variable for the function 
            val tfun = tCons.createFreshVar()
            println(s"Type Var: $funName has type variable $tfun")
            //3. Update the type environment.
            val newEnv = alpha ++ List(param -> tx, funName -> tfun)
            //4. Process the body of the function having updated the type environment
            val t2 = generateTypeVarsAndConstraints(funExpr, newEnv, tCons)
            //5. The type of the function better be tx => t2, tx being argument type and t2 being
            // what we computed as the type of its body
            tCons.addTypeEquation(FunType(tx, t2), tfun)
            // return the type of the function
            tfun
        }
    }
}

def generateAllEquations(p: Program ) = p match {
    case TopLevel(e) => {
        val tCons = new TypeConstraints()
        val emptyEnv: Map[String, Type] = Map()
        val t = generateTypeVarsAndConstraints(e, emptyEnv, tCons)
        println("Equations: ")
        tCons.printAllTypeEquations()
        println("------------- ")
        println(s"If equations are solved then the program will have type : $t")
        tCons
    }
}




defined function generateTypeVarsAndConstraints
defined function generateAllEquations

In [4]:
// let x = 15 in x + 35
val x = Ident("x")
val e1 = Let("x", Const(15), Plus(x, Const(35)))
val tCons1 = generateAllEquations(TopLevel(e1))

Debug: trivial type equation num == num suppressed.
Debug: trivial type equation num == num suppressed.
Equations: 
------------- 
If equations are solved then the program will have type : num


x: Ident = Ident("x")
e1: Let = Let(x,Const(15.0),Plus(Ident(x),Const(35.0)))
tCons1: TypeConstraints = TypeConstraints()

In [8]:
// let f = function (x)  x >= 35 in 
//   f(20) + 35

val f = Ident("f")
val x = Ident("x")
val fdef = FunDef("x", Geq(x, Const(35)))
val e2 = Let("f", fdef, Plus(FunCall(f, Const(20)), Const(35)))
val tCons2 = generateAllEquations(TopLevel(e2))

Type Var: x has type variable t0
Debug: trivial type equation num == num suppressed.
Debug: trivial type equation num == num suppressed.
Equations: 
bool == num
num == t0
t0 == num
------------- 
If equations are solved then the program will have type : num


f: Ident = Ident("f")
x: Ident = Ident("x")
fdef: FunDef = FunDef(x,Geq(Ident(x),Const(35.0)))
e2: Let = Let(f,FunDef(x,Geq(Ident(x),Const(35.0))),Plus(FunCall(Ident(f),Const(20.0)),Const(35.0)))
tCons2: TypeConstraints = TypeConstraints()

In [9]:
/* let f = function (x) x + x in 
  f (f)
*/
val f = Ident("f")
val x = Ident("x")
val fdef = FunDef("x", Plus(x, x))
val e3 = Let("f", fdef, FunCall(f, f))
val tCons3 = generateAllEquations(TopLevel(e3))

Type Var: x has type variable t0
Equations: 
(t0 => num) == t0
t0 == num
t0 == num
------------- 
If equations are solved then the program will have type : num


f: Ident = Ident("f")
x: Ident = Ident("x")
fdef: FunDef = FunDef(x,Plus(Ident(x),Ident(x)))
e3: Let = Let(f,FunDef(x,Plus(Ident(x),Ident(x))),FunCall(Ident(f),Ident(f)))
tCons3: TypeConstraints = TypeConstraints()

In [7]:
/* let f = function (g) 
             function (x) 
                 g( g( x ) )
         in 
      let d = function (y) y + y in 
          f (d) */

val d = Ident("d")
val y = Ident("y")
val g = Ident("g")
val x = Ident("x")
val f = Ident("f")
val fdef1 = FunDef("y", Plus(y, y))

val e1 = Let("d", fdef1, FunCall(f, d))
val fdef3 = FunDef("x", FunCall(g, FunCall(g, x)))
val fdef2 = FunDef("g", fdef3)
val e4  = Let("f", fdef2, e1)

val tCons4 = generateAllEquations(TopLevel(e4))

Type Var: g has type variable t0
Type Var: x has type variable t1
Type Var: FunCall(Ident(g),FunCall(Ident(g),Ident(x))) has type variable t2
Type Var: FunCall(Ident(g),Ident(x)) has type variable t3
Type Var: y has type variable t4
Equations: 
(t4 => num) == t0
t4 == num
t4 == num
t0 == (t3 => t2)
t0 == (t1 => t3)
------------- 
If equations are solved then the program will have type : (t1 => t2)


d: Ident = Ident("d")
y: Ident = Ident("y")
g: Ident = Ident("g")
x: Ident = Ident("x")
f: Ident = Ident("f")
fdef1: FunDef = FunDef(y,Plus(Ident(y),Ident(y)))
e1: Let = Let(d,FunDef(y,Plus(Ident(y),Ident(y))),FunCall(Ident(f),Ident(d)))
fdef3: FunDef = FunDef(x,FunCall(Ident(g),FunCall(Ident(g),Ident(x))))
fdef2: FunDef = FunDef(g,FunDef(x,FunCall(Ident(g),FunCall(Ident(g),Ident(x)))))
e4: Let = Let(f,FunDef(g,FunDef(x,FunCall(Ident(g),FunCall(Ident(g),Ident(x))))),Let(d,FunDef(y,Plus(Ident(y),Ident(y))),FunCall(Ident(f),Ident(d))))
tCons4: TypeConstraints = TypeConstraints()

In [10]:
/* let rec f = function (z) if (z == 0) then 1 else f(z -1) in f(10) */
 val p5 = TopLevel(LetRec("f", "z", IfThenElse( 
                         Geq(Const(0), Ident("z")),
                         Const(1),
                         Plus(Const(1), FunCall(Ident("f"), Minus(Ident("z"), Const(1))))
                         ), 
                  FunCall(Ident("f"), Const(10))
         ))
val tCons5 = generateAllEquations(p5)

Type Var: z has type variable t0
Type Var: f has type variable t1
Debug: trivial type equation num == num suppressed.
Debug: trivial type equation bool == bool suppressed.
Debug: trivial type equation num == num suppressed.
Type Var: FunCall(Ident(f),Minus(Ident(z),Const(1.0))) has type variable t2
Debug: trivial type equation num == num suppressed.
Debug: trivial type equation num == num suppressed.
Equations: 
(t0 => num) == t1
t2 == num
t1 == (num => t2)
t0 == num
t0 == num
------------- 
If equations are solved then the program will have type : t1


p5: TopLevel = TopLevel(LetRec(f,z,IfThenElse(Geq(Const(0.0),Ident(z)),Const(1.0),Plus(Const(1.0),FunCall(Ident(f),Minus(Ident(z),Const(1.0))))),FunCall(Ident(f),Const(10.0))))
tCons5: TypeConstraints = TypeConstraints()

## 2. Solving Constraints

We saw how to systematically generate constraints. The question is how do we solve them.
The algorithm thus far is to keep two parts: (a) substitutions and (b) remaining constraints.

The key data structure is that of a substitution.  We will implement a substitution is simply a map from a type variable to a RHS constraint. Let us start by writing some utility functions below.

In [12]:
/* Define a special exception for us to throw */
case class SolverError(msg: String) extends Exception

/* Write a function that returns true if a type contains a typevariable and false otherwise */
def typeExprContainsVariable(tExpr: Type, tVar: TypeVar): Boolean = tExpr match {
    case TypeVar(_) => tExpr == tVar // If we have a type variable, check if it is equal to what we are searching.
    //For function types, recursively search both parts
    case FunType(t1, t2) => (typeExprContainsVariable(t1, tVar) || typeExprContainsVariable(t2, tVar))
    // If we encounter anything else, just return false 
    case _ => false
}

/* Write a function to apply a substitution to a type tExpr.
tExpr is a type eg., (t1 => t2) => num 
subst is a subsitution eg., t2 |---> (num => num)
result will be (t1 => (num => num)) => num 

Note subst can have multiple type variables on the LHS at the same time.
*/

def substituteExpr(tExpr: Type, subst: Map[TypeVar, Type]): Type = tExpr match {
    /* -- If the type expression tExpr is just a variable, then we simply substitute it as in subst -- */
    case TypeVar(j) => if (subst contains (TypeVar(j))) 
                        subst(TypeVar(j))
                       else 
                        tExpr
    /*-- If the type expression is a function t1 => t2, recurse on t1, t2 --*/
    case FunType(t1, t2) => FunType( (substituteExpr(t1, subst)), (substituteExpr(t2, subst)) )
    /* If not, return tExpr unchanged */
    case _ => tExpr
}


defined class SolverError
defined function typeExprContainsVariable
defined function substituteExpr

Now we come to a basic function that will update an existing substitution `subst` with a new substitution rule  that  maps `tVar` to `tExpr`. 

However, note that `tVar` may occur in the RHS of expressions in `subst`. For instance, subst may be

$$\begin{array}{rcl}
t_1 & \mapsto & t_2 \Rightarrow t_3 \\
t_4 & \mapsto & \text{num} \Rightarrow \text{num} \\
\end{array}$$

And we wish to add a new rule $t_2 \mapsto \text{num} \Rightarrow t_3 $.

However, note that $t_1$s rule involves $t_2$ in it. Therefore, we first substitute in our existing right hand sides to convert the substitution into 

$$\begin{array}{rcl}
t_1 & \mapsto & \color{red}{(\text{num} \Rightarrow t_3)} \Rightarrow t_3 \\
t_4 & \mapsto & \text{num} \Rightarrow \text{num} \\
\end{array}$$
The final subsitution is
$$\begin{array}{rcl}
t_1 & \mapsto & {(\text{num} \Rightarrow t_3)} \Rightarrow t_3 \\
t_4 & \mapsto & \text{num} \Rightarrow \text{num} \\
\color{red}{t_2} & \mapsto &  \color{red}{\text{num} \Rightarrow t_3}\\
\end{array}$$

The code is simple: we use a foldLeft operator that starts with empty map and runs through
each rule $t \maspto te$ in the substitution map.
- It substitutes the RHS $te$ so that the new type variable `tVar` never appears in the RHS.
- It adds the modified substitution rule to the new map being built.
- Finally, at the very end, the new substitution rule `tVar -> tExpr` is also added 


In [13]:
def updateSubstitutionWithNewRule(tVar: TypeVar, tExpr: Type, subst: Map[TypeVar, Type] ): Map[TypeVar, Type] = {
    assert(! subst.contains(tVar), "Substitution already contains type variable. This cannot happen - your algorithm may be buggy")
    assert(! typeExprContainsVariable(tExpr, tVar), "Substitution RHS cannot contain LHS variable")
    val newSubst = subst.foldLeft[Map[TypeVar,Type]] (Map[TypeVar,Type]()) {
                          case (m, (t, te)) => { m + (t -> substituteExpr(te, Map(tVar -> tExpr)  )) }
                        }
    newSubst + (tVar -> tExpr)
}


defined function updateSubstitutionWithNewRule

Now, we describe the main algorithm for processing an equation $t_1 = t_2$. It boils down to many cases.

The function `processEquation` processes a single equation $\hat{t_1} == \hat{t_2}$ with a current substitution `subst`. The return value is a new substitution.

1. Substitute `t1Hat`, `t2Hat` according to the current substitution `subst`.
2. If the resulting types `t1, t2` are the same, we have nothing more to do.
3. Otherwise, we split on many cases, as you can see below in the code.
  - If they are incompatible types, throw a `SolverError` exception.
  - If not, if one of the expressions is a type variable, make it into a new substitution rule.
  - Otherwise, recursively unify. 
  
With this, the code should hopefully make more sense now.


In [14]:

def isFunctionType(t: Type) = t match {
    case FunType(_,_) => true
    case _ => false
}

/*-- processEquation has as inputs
 - teq:  an equation
 - subst: the current set of substitutions
 Returns
 new substitution
 --*/

def processEquation(teq: TypeEquation, subst: Map[TypeVar, Type], provenance: String = ""): Map[TypeVar, Type] =  {
    /* 1. Take the current equation and apply all possible substitutions to the LHS and RHS */
    val (t1, t2) = teq match {
        case TypeEquation(t1Hat, t2Hat) => ( substituteExpr(t1Hat, subst), substituteExpr(t2Hat, subst))
    }
    if (t1 == t2) {
        subst /* If result are same for LHS and RHS, nothing more to be done. */
    } else {
       (t1, t2) match {
           /* The unification of t1 and t2 begins here */
          
            case (NumType, BoolType) => {  /* t1, t2 cannot be unified */
                throw new SolverError(s"@ $provenance: Numerical and Boolean types used interchangably")
            }
           
            
            case (BoolType, NumType) => { /* t1, t2 cannot be unified */
                throw new SolverError(s"@ $provenance: Numerical and Boolean types used interchangably")
            }
           
            
            case (NumType, tf) if isFunctionType(tf) => {/* t1, t2 cannot be unified */
                 throw new SolverError( s"@ $provenance: Numerical and Function types used interchangably")
            }
           
           
            case (tf, NumType) if isFunctionType(tf) => { /* t1, t2 cannot be unified */
                 throw new SolverError(s"@ $provenance: Numerical and Function types used interchangably")
            }
            case (BoolType, tf) if isFunctionType(tf) => { /* t1, t2 cannot be unified */
                 throw new SolverError(s"@ $provenance: Numerical and Function types used interchangably")
            }
            case (tf, BoolType) if isFunctionType(tf) => { /* t1, t2 cannot be unified */
                 throw new SolverError(s"@ $provenance: Numerical and Function types used interchangably")
            }
           /*t1 is a type variable */
            case (TypeVar(j), _ ) =>{ /* t1 and t2 can be unified, if t2 does not contain the type variable t1 */
                if (typeExprContainsVariable(t2, TypeVar(j))){
                    throw new SolverError(s"@ $provenance: Type variable also appears in the RHS of an equation: no solution can exist.")
                }
                /* Add a new substitution rule */
                updateSubstitutionWithNewRule(TypeVar(j), t2, subst)
            } 
            /* Same as previous case except order reversed */
           case (_, TypeVar(j)) => {
               assert(t1 != TypeVar(j))
               if (typeExprContainsVariable(t1, TypeVar(j))){
                    throw new SolverError(s"@ $provenance: Type variable also appears in the RHS of an equation: no solution can exist.")
                }
               updateSubstitutionWithNewRule(TypeVar(j), t1, subst)
           }
           /* t1 and t2 are both function types */
           case (FunType(t1Hat, t2Hat), FunType(t3Hat, t4Hat)) => {
               /* Unify the two parts of the function type recursively */
               val subst1 = processEquation(TypeEquation(t1Hat, t3Hat), subst, provenance)
               val subst2 = processEquation(TypeEquation(t2Hat, t4Hat), subst1, provenance)
               subst2
           }
           
           case _ => {
               throw new SolverError(s"@ $provenance:Cannot unify disparate types $t1, $t2")
           }
       }
    }
}


def processAllEquations(tCons: TypeConstraints) = {
    println("Equations: ")
    tCons.printAllTypeEquations()
    println("Solving:")
    val finalSubst = tCons.l.foldLeft[Map[TypeVar, Type]] (Map()) {
        case (subst, te1) => processEquation(te1, subst)
    }
    println(s"Solution: $finalSubst")
}

defined function isFunctionType
defined function processEquation
defined function processAllEquations

In [15]:
processAllEquations(tCons1)

Equations: 
Solving:
Solution: Map()


In [16]:
try { processAllEquations(tCons2) } catch {case SolverError(msg) => println(s"ERROR: $msg")}

Equations: 
bool == num
num == t0
t0 == num
Solving:
ERROR: @ : Numerical and Boolean types used interchangably


In [17]:
try {processAllEquations(tCons3)} catch { case SolverError(msg) => println(s"ERROR: $msg")}

Equations: 
(t0 => num) == t0
t0 == num
t0 == num
Solving:
ERROR: @ : Type variable also appears in the RHS of an equation: no solution can exist.


In [18]:
try processAllEquations(tCons4) catch { case e => print(e)}

Equations: 
(t4 => num) == t0
t4 == num
t4 == num
t0 == (t3 => t2)
t0 == (t1 => t3)
Solving:
Solution: Map(t0 -> (num => num), t4 -> num, t1 -> num, t3 -> num, t2 -> num)


## Type Inference In One Shot

Instead of first generating and then solving type constraints, why not try to do things in __one shot__? 

We can achieve this and it is not all that complicated: 
- Instead of collecting constraints, every time we create an equality, process it using the 
Simply carry around the current substitution as we are solving constraints.

In [19]:
/* -- We need something global in scope for managing fresh type variables. 
    There are other ways to achieve this but let us suck it up and use the most 
    convenient way for now.
    --*/

object TypeVars { 
    var numTypeVars = 0
    def createFreshVar(): TypeVar = {
        val t = TypeVar(numTypeVars)
        numTypeVars = numTypeVars + 1
        return t
    }
}


def typeInferExpression(e: Expr, alpha: Map[String, Type], subst: Map[TypeVar, Type]): (Type, Map[TypeVar, Type]) = {
    /* 
      e is the expression we are processing.
      alpha is the current type environment
      tCons (MUTABLE) stores the type constraints thus far
      
      Returns
        Nothing (since tCons is mutable)
   */
    def handleSubExprs(e1: Expr, e2: Expr, operandType: Type, resultType: Type, pWhere: String): (Type, Map[TypeVar, Type]) = {
            val (t1, subst1) = typeInferExpression(e1, alpha, subst) 
            val subst2 = processEquation( TypeEquation(t1, operandType), subst1, pWhere + "(first argument)")
            val (t2, subst3) = typeInferExpression(e2, alpha, subst2)
            val subst4 = processEquation( TypeEquation(t2, operandType), subst3, pWhere + "(second argument)")
            (resultType, subst4)
    }
    
    
    e match {
        case Const(_) => (NumType, subst)
        case Ident(s) => if (alpha contains s) 
                            (alpha(s), subst)
                         else 
                            throw new SolverError(s"Unknown identifier: $s")
        case Minus(e1, e2) => handleSubExprs(e1, e2, NumType, NumType,  "- must operate on numbers")
         
        case Plus(e1, e2) => handleSubExprs(e1, e2, NumType, NumType, "+ must operate on numbers")
        
        case Mult(e1, e2) => handleSubExprs(e1, e2, NumType, NumType, "* must operate on numbers")
        
        case Geq(e1, e2) => handleSubExprs(e1, e2, NumType,  BoolType,  ">= must operate on numbers")
        
        case Eq(e1, e2) => {
                val (t1, s1) = typeInferExpression(e1, alpha, subst)
                val (t2, s2) = typeInferExpression(e2, alpha, s1)
                val s3 = processEquation( TypeEquation(t1, t2), s2, "== arguments must have same type")
                (BoolType, s3)   
            }
        
        
        case IfThenElse(e, e1, e2) => { 
                val (t, s1) = typeInferExpression(e, alpha, subst)    
                val s2 =  processEquation( TypeEquation(t, BoolType), s1, "If Then Else condition must be boolean") 
                val (t1, s3) = typeInferExpression(e1, alpha, s2)
                val (t2, s4) = typeInferExpression(e2, alpha, s3)
                val s5 = processEquation( TypeEquation(t1, t2), s4, "If Then Else both branches must be the same type")
                (t1, s5)
        }
        
        case Let(x, e1, e2) => {
            val (t1, s1) = typeInferExpression(e1, alpha, subst)
            val newEnv = alpha + (x -> t1)
            typeInferExpression(e2, newEnv, s1)
        }
        
        case FunCall(e1, e2) => {
            val (t, s1) = typeInferExpression(e1, alpha, subst)
            t match {
                case FunType(t1Hat , t2Hat) => {
                    val (t1, s2) = typeInferExpression(e2, alpha, s1)
                    val s3 = processEquation( TypeEquation(t1, t1Hat), s2, "Function call argument type mismatch")
                    (t2Hat, s3)
                }
                case TypeVar(j) => {
                    val nVar = TypeVars.createFreshVar() // Create a placeholder for the result type
                    println(s"Type Var: $e has type variable $nVar")
                    val (t1, s1) = typeInferExpression(e2, alpha, subst)
                    val t3 = FunType(t1, nVar)
                    val s2 = processEquation( TypeEquation(t, t3), s1, "Function call type mismatch ")
                    (nVar, s2)
                }
                case _ => throw new SolverError(s"Type inference error: function call on non function type $t")
            }
           
        }
        
        case FunDef(x, e1) => {
            //1. Create a new variable for the formal argument x
            val tx = TypeVars.createFreshVar()
            println(s"Type Var: $x has type variable $tx")
            //2. Add it to the type environment
            val newEnv = alpha + (x -> tx)
            val (t2, s2) = typeInferExpression(e1, newEnv, subst)
            //3. Type of this whole thing is tx => t2
            (FunType(tx, t2), s2)
        }
        
        case LetRec(funName, param, funExpr, bodyExpr) => {
            //1. Create a new variable for the formal argument x
            val tx = TypeVars.createFreshVar()
            println(s"Type Var: $param has type variable $tx")
            val tfun = TypeVars.createFreshVar()
            println(s"Type Var: $funName has type variable $tfun")
            val newEnv = alpha ++ List(param -> tx, funName -> tfun)
            val (t2, s2) = typeInferExpression(funExpr, newEnv, subst)
            val s3 = processEquation(TypeEquation(FunType(tx, t2), tfun), s2, "Recursive function is not well typed")
            (tfun, s3)
        }
    }
}

def typeInferProgram(p: Program ) = p match {
    case TopLevel(e) => {
        val subst: Map[TypeVar, Type] = Map()
        val emptyEnv: Map[String, Type] = Map()
        try {
            val (t, sFinal) = typeInferExpression(e, emptyEnv, subst)
            println(s"Type Checked: Program will have type : ${substituteExpr(t, sFinal)}")
        } catch {
            case SolverError(msg) => {println(s"FAILED: $msg")}
        }
    }
}



defined object TypeVars
defined function typeInferExpression
defined function typeInferProgram

In [20]:
val x = Ident("x")
val e1 = Let("x", Const(15), Plus(x, Const(35)))
typeInferProgram(TopLevel(e1))

Type Checked: Program will have type : num


x: Ident = Ident("x")
e1: Let = Let(x,Const(15.0),Plus(Ident(x),Const(35.0)))

In [22]:
// let f = function(x) x >= 35 in 
// .   f(20 ) + 35

val f = Ident("f")
val x = Ident("x")
val fdef = FunDef("x", Geq(x, Const(35)))
val e2 = Let("f", fdef, Plus(FunCall(f, Const(20)), Const(35)))
typeInferProgram(TopLevel(e2))

Type Var: x has type variable t1
FAILED: @ + must operate on numbers(first argument): Numerical and Boolean types used interchangably


f: Ident = Ident("f")
x: Ident = Ident("x")
fdef: FunDef = FunDef(x,Geq(Ident(x),Const(35.0)))
e2: Let = Let(f,FunDef(x,Geq(Ident(x),Const(35.0))),Plus(FunCall(Ident(f),Const(20.0)),Const(35.0)))

In [23]:
/* let f = function (x) x + x in 
  f (f)
*/
val f = Ident("f")
val x = Ident("x")
val fdef = FunDef("x", Plus(x, x))
val e3 = Let("f", fdef, FunCall(f, f))
typeInferProgram(TopLevel(e3))

Type Var: x has type variable t2
FAILED: @ Function call argument type mismatch: Numerical and Function types used interchangably


f: Ident = Ident("f")
x: Ident = Ident("x")
fdef: FunDef = FunDef(x,Plus(Ident(x),Ident(x)))
e3: Let = Let(f,FunDef(x,Plus(Ident(x),Ident(x))),FunCall(Ident(f),Ident(f)))

In [24]:
/* let f = function (g) 
             function (x) 
                 g( g( x ) )
         in 
      let d = function (y) y + y in 
          f (d) */

val d = Ident("d")
val y = Ident("y")
val g = Ident("g")
val x = Ident("x")
val f = Ident("f")
val fdef1 = FunDef("y", Plus(y, y))

val e1 = Let("d", fdef1, FunCall(f, d))
val fdef3 = FunDef("x", FunCall(g, FunCall(g, x)))
val fdef2 = FunDef("g", fdef3)
val e4  = Let("f", fdef2, e1)
typeInferProgram(TopLevel(e4))

Type Var: g has type variable t3
Type Var: x has type variable t4
Type Var: FunCall(Ident(g),FunCall(Ident(g),Ident(x))) has type variable t5
Type Var: FunCall(Ident(g),Ident(x)) has type variable t6
Type Var: y has type variable t7
Type Checked: Program will have type : (num => num)


d: Ident = Ident("d")
y: Ident = Ident("y")
g: Ident = Ident("g")
x: Ident = Ident("x")
f: Ident = Ident("f")
fdef1: FunDef = FunDef(y,Plus(Ident(y),Ident(y)))
e1: Let = Let(d,FunDef(y,Plus(Ident(y),Ident(y))),FunCall(Ident(f),Ident(d)))
fdef3: FunDef = FunDef(x,FunCall(Ident(g),FunCall(Ident(g),Ident(x))))
fdef2: FunDef = FunDef(g,FunDef(x,FunCall(Ident(g),FunCall(Ident(g),Ident(x)))))
e4: Let = Let(f,FunDef(g,FunDef(x,FunCall(Ident(g),FunCall(Ident(g),Ident(x))))),Let(d,FunDef(y,Plus(Ident(y),Ident(y))),FunCall(Ident(f),Ident(d))))

In [25]:
/* let rec f = function (z) if (z <= 0) then 1 else f(z -1) in f(10) */
 val p5 = TopLevel(LetRec("f", "z", IfThenElse( 
                         Geq(Const(0), Ident("z")),
                         Const(1),
                         Plus(Const(1), FunCall(Ident("f"), Minus(Ident("z"), Const(1))))
                         ), 
                  FunCall(Ident("f"), Const(10))
         ))
typeInferProgram(p5)

Type Var: z has type variable t8
Type Var: f has type variable t9
Type Var: FunCall(Ident(f),Minus(Ident(z),Const(1.0))) has type variable t10
Type Checked: Program will have type : (num => num)


p5: TopLevel = TopLevel(LetRec(f,z,IfThenElse(Geq(Const(0.0),Ident(z)),Const(1.0),Plus(Const(1.0),FunCall(Ident(f),Minus(Ident(z),Const(1.0))))),FunCall(Ident(f),Const(10.0))))